In [2]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchinfo import summary
import joblib
import pennylane as qml
torch.manual_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
validation_data=pd.read_csv('./assets/processed dataset/validation dataset.csv')
test_data=pd.read_csv('./assets/processed dataset/test dataset.csv')
train_data=pd.read_csv('./assets/processed dataset/train dataset.csv')
scaler =joblib.load('./assets/scaler.gz')

In [5]:
df1=test_data
input_test=scaler.transform(df1)
input_test[0]

array([0.        , 0.25459318, 0.43522505, 0.29502573])

In [6]:
seq_length=180
#Getting x and y for testing
X_test=[]
Y_test=[]

for i in range(seq_length,input_test.shape[0]):
    #Input features from 0 to 59
    X_test.append(input_test[i-seq_length:i])
    #T2M at 60
    Y_test.append(input_test[i])

# converting into numpy arrays
X_test,Y_test=np.array(X_test),np.array(Y_test)
Y_test=Y_test.reshape(-1,4)
print(X_test.shape,Y_test.shape)

X_test = Variable(torch.Tensor(X_test))
Y_test = Variable(torch.Tensor(Y_test))
print(X_test.shape,Y_test.shape)

(1957, 180, 4) (1957, 4)
torch.Size([1957, 180, 4]) torch.Size([1957, 4])


In [7]:
#Defining quantum layer
n_qubits = 4
n_layers=6
weight_shapes = {"weights": (n_layers, n_qubits)}
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev,interface='torch')
def circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    exp_val=[qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]
    return tuple(exp_val)
    #return exp_val

# qml.drawer.use_style("black_white")
# fig,ax=qml.draw_mpl(circuit)([0.26,0.28,3.31,0.85],torch.randn(6,4))
# plt.show()
qlayer = qml.qnn.TorchLayer(circuit, weight_shapes)
qlayer

<Quantum Torch Layer: func=circuit>

In [8]:
#HYBRID QLSTM Network definition
class NETWORK_HYBRID_BILSTM(nn.Module):

    def __init__(self,num_classes,input_size,hidden_size,num_layers):
        super(NETWORK_HYBRID_BILSTM,self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.lstm1=nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True,bidirectional=True).to(device)
        
        self.fc1=nn.Linear(hidden_size*2,n_qubits)
        self.qlayer=qlayer
        self.fc2=nn.Linear(n_qubits,num_classes)
    
    def forward(self,x):
        output_lstm1, (h_n,c_n) = self.lstm1(x)
        output_fc1=self.fc1(output_lstm1[:,-1,:])
        input_q=output_fc1        
        output_q=self.qlayer(input_q)
        output_fc2=self.fc2(output_q).to(device)
        return output_fc2

In [9]:
input_size=4
hidden_size=8
hidden_size2=8
num_layers=1
num_classes=4

model=NETWORK_HYBRID_BILSTM(num_classes,input_size, hidden_size,num_layers)
model.load_state_dict(torch.load('./assets/trained models state/hybrid_bilstm_state.pt'))
model.eval()


NETWORK_HYBRID_BILSTM(
  (lstm1): LSTM(4, 8, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=16, out_features=4, bias=True)
  (qlayer): <Quantum Torch Layer: func=circuit>
  (fc2): Linear(in_features=4, out_features=4, bias=True)
)

In [10]:
#Testing model
Y_pred=model(X_test.to(device))
Y_pred=Y_pred.detach().numpy()

In [11]:
#MSE
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(Y_test,Y_pred)
mse

0.008305465

In [12]:
from sktime.performance_metrics.forecasting import mean_absolute_scaled_error
mase=mean_absolute_scaled_error(Y_test,Y_pred,y_train=Y_test)
mase

2.1676287878433027